# Mosaic Quickstart (CPU)

This notebook demonstrates a minimal, CPU-friendly protein design loop using [escalante-bio/mosaic](https://github.com/escalante-bio/mosaic). We optimize a short sequence with a trigram language-model objective and decode the design.


In [ ]:
# If running in Colab, uncomment the next line to install Mosaic
%pip -q install "git+https://github.com/escalante-bio/mosaic"


In [ ]:
import numpy as np
import jax.numpy as jnp
from mosaic.losses import TrigramLL
from mosaic.optimizers import simplex_APGM

# Reproducibility
np.random.seed(0)

length = 35
num_aas = 20
prob0 = jnp.full((length, num_aas), 1.0 / num_aas)

trigram = TrigramLL.from_pkl()

def objective(p):
    return trigram(p)  # negative log-likelihood

history = simplex_APGM(
    objective, x0=prob0, steps=400, stepsize=0.1 * (length ** 0.5), scale=1.0, logspace=True
)

alphabet = 'ACDEFGHIKLMNPQRSTVWY'
x_final = np.array(history['x'][-1])
seq = ''.join(alphabet[i] for i in np.argmax(x_final, axis=1))
print('Designed sequence:', seq)


In [ ]:
# Optional: visualize convergence
import matplotlib.pyplot as plt
plt.plot(history['loss'], label='total')
plt.legend(); plt.xlabel('step'); plt.ylabel('loss'); plt.show()
